In [2]:
import pandas as pd
# If put the data(.csv) under the same folder, you could use
df = pd.read_csv('garments_worker_productivity.csv')

In [3]:
columns=list(df.columns[1:])
df=df[columns]
df['day']=df['day'].transform(lambda x: x.strip())
df['department']=df['department'].transform(lambda x: 'sewing' if x=='sweing' else x.strip())
df['satisfied']= df['actual_productivity']>=df['targeted_productivity']
df=df.drop(["actual_productivity", "targeted_productivity"], axis=1)
df['wip']=df['wip'].fillna(0)

In [4]:
df['day']=df['day'].transform(lambda x: 0 if x == 'Thursday' else 1 if x == 'Saturday' else 2 if x == 'Sunday' else 3 if x == 'Monday' else 4 if x == 'Tuesday' else 5)
df['quarter']=df['quarter'].transform(lambda x: 0 if x=='Quarter1' else 1 if x=='Quarter2' else 2 if x=='Quarter3' else 3 if x=='Quarter4' else 4)
df['department']=df['department'].transform(lambda x: 0 if x=='sewing' else 1)
poss=list(df['team'].unique())
df['team']=df['team'].transform(lambda x: poss.index(x))
train_df = df.iloc[:int(len(df) * 0.8)]
test_df  = df.iloc[int(len(df) * 0.8):] 
X=train_df[['quarter','department','day','team','satisfied']]
P_s1=X[X['satisfied']==True].shape[0]/(X['satisfied'].shape[0])
P_s0=X[X['satisfied']==False].shape[0]/(X['satisfied'].shape[0])

In [5]:
X= train_df[list(train_df.columns)[4:]]
X

,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,satisfied
0,26.16,1108.0,7080,98,0.0,0,0,59.0,True
1,3.94,0.0,960,0,0.0,0,0,8.0,True
2,11.41,968.0,3660,50,0.0,0,0,30.5,True
3,11.41,968.0,3660,50,0.0,0,0,30.5,True
4,25.90,1170.0,1920,50,0.0,0,0,56.0,True
...,...,...,...,...,...,...,...,...,...
952,29.40,916.0,6960,56,0.0,0,2,58.0,True
953,21.82,1591.0,3240,0,0.0,0,1,52.0,True
954,30.33,398.0,6960,0,0.0,0,1,58.0,False
955,4.60,0.0,3780,0,0.0,0,0,9.0,False


In [6]:
temp= X.iloc[:,:-1]
temp

,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers
0,26.16,1108.0,7080,98,0.0,0,0,59.0
1,3.94,0.0,960,0,0.0,0,0,8.0
2,11.41,968.0,3660,50,0.0,0,0,30.5
3,11.41,968.0,3660,50,0.0,0,0,30.5
4,25.90,1170.0,1920,50,0.0,0,0,56.0
...,...,...,...,...,...,...,...,...
952,29.40,916.0,6960,56,0.0,0,2,58.0
953,21.82,1591.0,3240,0,0.0,0,1,52.0
954,30.33,398.0,6960,0,0.0,0,1,58.0
955,4.60,0.0,3780,0,0.0,0,0,9.0


In [7]:
X['satisfied'].unique()

array([ True, False])

In [8]:
import numpy as np

In [9]:
my_dict={}
for i in train_df['satisfied'].unique():
    my_dict[i]= {'mean':{i: temp[i].mean() for i in temp.columns}, 'std':{i: temp[i].mean() for i in temp.columns}}
my_dict

{np.True_: {'mean': {'smv': np.float64(15.25889237199582),
   'wip': np.float64(732.7523510971787),
   'over_time': np.float64(4751.755485893417),
   'incentive': np.float64(26.734587251828632),
   'idle_time': np.float64(0.8944618599791013),
   'idle_men': np.float64(0.35214211076280044),
   'no_of_style_change': np.float64(0.13479623824451412),
   'no_of_workers': np.float64(35.1128526645768)},
  'std': {'smv': np.float64(15.25889237199582),
   'wip': np.float64(732.7523510971787),
   'over_time': np.float64(4751.755485893417),
   'incentive': np.float64(26.734587251828632),
   'idle_time': np.float64(0.8944618599791013),
   'idle_men': np.float64(0.35214211076280044),
   'no_of_style_change': np.float64(0.13479623824451412),
   'no_of_workers': np.float64(35.1128526645768)}},
 np.False_: {'mean': {'smv': np.float64(15.25889237199582),
   'wip': np.float64(732.7523510971787),
   'over_time': np.float64(4751.755485893417),
   'incentive': np.float64(26.734587251828632),
   'idle_time'

In [10]:
prior={}
for i in train_df['satisfied'].unique():
    prior[i]= X[X['satisfied']==i].shape[0]/X.shape[0]
prior

{np.True_: 0.7314524555903866, np.False_: 0.26854754440961337}

In [11]:
X= test_df[list(test_df.columns)[4:]]
temp2= X.iloc[:,:-1]
temp2

,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers
957,30.10,437.0,7080,32,0.0,0,2,59.0
958,27.48,413.0,6840,38,0.0,0,1,57.0
959,2.90,0.0,3360,0,0.0,0,0,8.0
960,2.90,0.0,960,0,0.0,0,0,8.0
961,26.66,1164.0,6600,23,0.0,0,2,55.0
...,...,...,...,...,...,...,...,...
1192,2.90,0.0,960,0,0.0,0,0,8.0
1193,3.90,0.0,960,0,0.0,0,0,8.0
1194,3.90,0.0,960,0,0.0,0,0,8.0
1195,2.90,0.0,1800,0,0.0,0,0,15.0


In [12]:
predictions = []
for i in range(test_df.shape[0]):
    posteriors = {}
    for j in prior.keys():  # j = True or False
        post = np.log(prior[j])  # prior probability log
        likelihood = 0
        for feature, x in temp2.iloc[i].items():
            mean = my_dict[j]['mean'][feature]
            std = my_dict[j]['std'][feature]
            epsilon = 1e-9
            # Gaussian likelihood
            coeff = 1.0 / (np.sqrt(2 * np.pi) * (std + epsilon))
            val = coeff * np.exp(-((x - mean) ** 2) / (2 * (std ** 2 + epsilon)))
            likelihood += np.log(val + 1e-9)
        posteriors[j] = post + likelihood
    predictions.append(max(posteriors, key=posteriors.get))

predictions = np.array(predictions)

In [13]:
predictions

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [ ]:
import math
class SVM: # Defaults to linear
    """
    Linear SVM trained by a simplified SMO algorithm on the dual.
    - Works for binary labels y in {-1, +1}
    - Soft margin when C is finite
    - Hard margin approximated with a very large C
    - No kernels (purely linear)

    After fit():
      - self.alpha: Lagrange multipliers (shape [m])
      - self.w, self.b: primal params with w = sum_i alpha_i y_i x_i
      - self.support_indices_: indices with alpha_i > tol
      - self.margin_sv_: indices with 0 < alpha_i < C - tol  (only meaningful if C < inf)
      - self.error_sv_: indices with alpha_i >= C - tol       (only meaningful if C < inf)
    """

    def __init__(self, C=1.0, tol=1e-4, eps=1e-6, max_passes=10, random_state=0):
        """
        C: soft-margin penalty. Use None or "hard" to approximate hard margin.
        tol: KKT tolerance for selecting violators.
        eps: minimal alpha change threshold within an SMO step.
        max_passes: SMO stopping criterion: consecutive passes with no alpha updates.
        """
        if C is None or (isinstance(C, str) and C.lower() == "hard"):
            # Large C approximates hard-margin constraints (requires separable data)
            C = 1e12
        self.C = float(C)
        self.tol = float(tol)
        self.eps = float(eps)
        self.max_passes = int(max_passes)
        self.random_state = int(random_state)

        # Learned params (set in the fit function)
        self.alpha = None
        self.b = 0.0
        self.w = None   # only for linear kernel
        # You will need to implement the following in your code
        self.alphas = None
        self.support_vectors = None
        self.support_vector_labels = None
        self.support_vector_alphas = None

        # Bookkeeping
        self.support_indices_ = None
        self.margin_sv_ = None
        self.error_sv_ = None


    def fit(self, X, y):
        """
        X: (m, n) numpy array
        y: (m,) labels in {-1, +1}
        """
        m, n = X.shape
        rng = np.random.RandomState(self.random_state)

        # Precompute kernel matrix for efficiency (linear)
        K = X @ X.T

        # Initialize dual variables
        alpha = np.zeros(m, dtype=float)
        b = 0.0

        # Error cache: E_i = f(x_i) - y_i
        def f(i):
            # f(x_i) = sum_j alpha_j y_j K_ji + b
            return float(((alpha * y)@ K[:, i]) + b)

        def E(i):
            return f(i) - y[i]

        passes = 0
        while passes < self.max_passes:
            num_changed = 0

            for i in range(m):
                Ei = (float(((alpha * y)@ K[:, i]) + b))- y[i]
                # KKT violation check (for selection), per standard SMO logic
                # Conditions for optimizing i:
                # (y_i * f_i < 1 - tol and alpha_i < C) or
                # (y_i * f_i > 1 + tol and alpha_i > 0)
                yi_fi = y[i] * (float(((alpha * y)@ K[:, i]) + b))
                if ((yi_fi < 1 - self.tol and alpha[i] < self.C) or
                    (yi_fi > 1 + self.tol and alpha[i] > 0)):

                    # Pick j != i (heuristic: random, you could implement max |Ei - Ej|)
                    j = i
                    while j == i:
                        j = rng.randint(0, m)
                    Ej = (float(((alpha * y)@ K[:, j]) + b))- y[j]

                    alpha_i_old, alpha_j_old = alpha[i], alpha[j]

                    # Compute L and H (box constraints)
                    if y[i] != y[j]:
                        L = max(0.0, alpha[j] - alpha[i])
                        H = min(self.C, self.C + alpha[j] - alpha[i])
                    else:
                        L = max(0.0, alpha[i] + alpha[j] - self.C)
                        H = min(self.C, alpha[i] + alpha[j])

                    if abs(L - H) < 1e-15:
                        continue

                    # eta = 2 * K_ij - K_ii - K_jj
                    eta = 2.0 * K[i, j] - K[i, i] - K[j, j]
                    if eta >= 0:
                        # Non-positive definiteness in this 2D subproblem; skip
                        continue

                    # Update alpha_j
                    alpha[j] = alpha_j_old - (y[j] * (Ei - Ej)) / eta
                    # Clip
                    if alpha[j] > H:
                        alpha[j] = H
                    elif alpha[j] < L:
                        alpha[j] = L

                    if abs(alpha[j] - alpha_j_old) < self.eps:
                        alpha[j] = alpha_j_old
                        continue

                    # Update alpha_i accordingly
                    alpha[i] = alpha_i_old + y[i] * y[j] * (alpha_j_old - alpha[j])

                    # Compute b1, b2 and update b
                    b1 = (b - Ei
                          - y[i] * (alpha[i] - alpha_i_old) * K[i, i]
                          - y[j] * (alpha[j] - alpha_j_old) * K[i, j])
                    b2 = (b - Ej
                          - y[i] * (alpha[i] - alpha_i_old) * K[i, j]
                          - y[j] * (alpha[j] - alpha_j_old) * K[j, j])

                    # Choose new b (per Platt)
                    if 0 < alpha[i] < self.C:
                        b = b1
                    elif 0 < alpha[j] < self.C:
                        b = b2
                    else:
                        b = 0.5 * (b1 + b2)

                    num_changed += 1

            if num_changed == 0:
                passes += 1
            else:
                passes = 0

        # Store learned parameters
        self.alpha = alpha
        self.b = float(b)
        # Primal weight vector w = sum_i alpha_i y_i x_i
        self.w = (alpha * y) @ X

        # Identify support vectors formally via alpha
        tol = 1e-8
        self.support_indices_ = np.where(alpha > tol)[0]
        if math.isfinite(self.C):
            self.margin_sv_ = np.where((alpha > tol) & (alpha < self.C - tol))[0]
            self.error_sv_ = np.where(alpha >= self.C - tol)[0]
        else:
            # Shouldn't occur because we map "hard" to large finite C above
            self.margin_sv_ = self.support_indices_
            self.error_sv_ = np.array([], dtype=int)

        return self

    def decision_function(self, X):
        return X @ self.w + self.b

    def predict(self, X):
        return np.where((X @ self.w + self.b) >= 0.0, 1.0, -1.0)

    def kernel_rbf(x1, x2, gamma=None):
        """Radial Basis Function (Gaussian) kernel"""
        if gamma is None:
            gamma = 1.0 / x1.shape[0]   # default: 1 / number of features
        return math.exp(-gamma * ((x1 - x2)@ (x1 - x2)))


In [ ]:
import numpy as np
import math

class SVM: # Defaults to linear
    def __init__(self, C=1.0, tol=1e-4, eps=1e-6, max_passes=10, random_state=0):
        if C is None or (isinstance(C, str) and C.lower() == "hard"):
            C = 1e12
        self.C = float(C)
        self.tol = float(tol)
        self.eps = float(eps)
        self.max_passes = int(max_passes)
        self.random_state = int(random_state)

        self.alpha = None
        self.b = 0.0
        self.w = None

        self.support_indices_ = None
        self.margin_sv_ = None
        self.error_sv_ = None

    def _linear_kernel(self, X, Z):
        return X @ Z.T

    def fit(self, X, y):

        m, n = X.shape
        rng = np.random.RandomState(self.random_state)

        # Precompute kernel
        K = X @ X.T

        alpha = np.zeros(m, dtype=float)
        b = 0.0

        f = (alpha * y) @ K + b  
        E = f - y

        passes = 0
        while passes < self.max_passes:
            num_changed = 0

            # Vectorized KKT check for all points
            yi_f = y * f
            violators = np.where(((yi_f < 1 - self.tol) & (alpha < self.C)) |
                                 ((yi_f > 1 + self.tol) & (alpha > 0)))[0]

            if len(violators) == 0:
                passes += 1
                continue

            for i in violators:
                Ei = E[i]

                # Pick j with heuristic: max |Ei - Ej|
                j = np.argmax(np.abs(E - Ei))
                if j == i:
                    continue

                Ej = E[j]
                alpha_i_old, alpha_j_old = alpha[i], alpha[j]

                # Compute L and H
                if y[i] != y[j]:
                    L = max(0.0, alpha[j] - alpha[i])
                    H = min(self.C, self.C + alpha[j] - alpha[i])
                else:
                    L = max(0.0, alpha[i] + alpha[j] - self.C)
                    H = min(self.C, alpha[i] + alpha[j])

                if abs(L - H) < 1e-15:
                    continue

                # eta
                eta = 2.0 * K[i, j] - K[i, i] - K[j, j]
                if eta >= 0:
                    continue

                # Update alpha_j
                alpha[j] = alpha_j_old - (y[j] * (Ei - Ej)) / eta
                alpha[j] = np.clip(alpha[j], L, H)

                if abs(alpha[j] - alpha_j_old) < self.eps:
                    alpha[j] = alpha_j_old
                    continue

                # Update alpha_i
                alpha[i] = alpha_i_old + y[i] * y[j] * (alpha_j_old - alpha[j])

                # Update b (vectorized rule)
                b1 = (b - Ei
                      - y[i] * (alpha[i] - alpha_i_old) * K[i, i]
                      - y[j] * (alpha[j] - alpha_j_old) * K[i, j])
                b2 = (b - Ej
                      - y[i] * (alpha[i] - alpha_i_old) * K[i, j]
                      - y[j] * (alpha[j] - alpha_j_old) * K[j, j])

                if 0 < alpha[i] < self.C:
                    b = b1
                elif 0 < alpha[j] < self.C:
                    b = b2
                else:
                    b = 0.5 * (b1 + b2)
                    
                delta_ai = alpha[i] - alpha_i_old
                delta_aj = alpha[j] - alpha_j_old
                E += (y[i] * delta_ai) * K[:, i] + (y[j] * delta_aj) * K[:, j] + (b - b1)

                # Update f_all too
                f = E + y

                num_changed += 1

            if num_changed == 0:
                passes += 1
            else:
                passes = 0

        # Store learned params
        self.alpha = alpha
        self.b = float(b)
        self.w = (alpha * y) @ X

        tol = 1e-8
        self.support_indices_ = np.where(alpha > tol)[0]
        if math.isfinite(self.C):
            self.margin_sv_ = np.where((alpha > tol) & (alpha < self.C - tol))[0]
            self.error_sv_ = np.where(alpha >= self.C - tol)[0]
        else:
            self.margin_sv_ = self.support_indices_
            self.error_sv_ = np.array([], dtype=int)

        return self

    def decision_function(self, X):
        return X @ self.w + self.b

    def predict(self, X):
        return np.where((X @ self.w + self.b) >= 0.0, 1.0, -1.0)